# 🌿 GreenSVC-AI Stage 1: Evidence-Based Indicator Matching

**Required Files**:
- `Evidence_final_v5_2_fixed.json` - Research evidence records
- `Appendix_final_v5_2_fixed.json` - Codebook (code definitions)

**Output**: Recommended indicators with evidence support

## 1. Environment Setup

In [ ]:
!pip install -q google-generativeai

In [ ]:
import os, json, re
from pathlib import Path
from datetime import datetime
from collections import defaultdict
from google.colab import drive, userdata, files
import google.generativeai as genai
print("✅ Libraries imported successfully")

✅ Libraries imported successfully


In [ ]:
drive.mount('/content/drive')
print("✅ Google Drive mounted")

Mounted at /content/drive
✅ Google Drive mounted


In [ ]:
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    print("✅ API Key retrieved from Secrets")
except:
    GOOGLE_API_KEY = input("Enter Google API Key: ")
genai.configure(api_key=GOOGLE_API_KEY)

## 2. Configuration

In [ ]:
CONFIG = {
    # 必需文件
    "evidence_path": "/content/drive/MyDrive/GreenSVC-AI-paper/KnowledgeBase/Evidence_final_v5_2_fixed.json",
    "appendix_path": "/content/drive/MyDrive/GreenSVC-AI-paper/KnowledgeBase/Appendix_final_v5_2_fixed.json",

    # 用户问卷
    "user_query_path": "/content/drive/MyDrive/GreenSVC-AI-paper/UserQueries/GreenSVC-AI_mock_filled_query_single_performance_photos_45_per_zone.json",

    # 输出
    "output_path": "/content/drive/MyDrive/GreenSVC-AI-paper/Outputs",

    # 模型
    "model_name": "gemini-3-pro-preview",
    "max_codebook_chars": 40000,
}
print(f"⚙️ Model: {CONFIG['model_name']}")

⚙️ Model: gemini-3-pro-preview


## 3. Load Data

In [ ]:
class KnowledgeBase:
    """加载和索引Evidence数据"""

    def __init__(self, evidence_path):
        self.evidence = []
        self.perf_idx = defaultdict(list)      # 按dimension索引
        self.subdim_idx = defaultdict(list)    # 按subdimension索引

        if Path(evidence_path).exists():
            self.evidence = json.load(open(evidence_path, encoding='utf-8'))
            print(f"📚 Evidence loaded: {len(self.evidence)} records")
            self._build_index()

    def _build_index(self):
        """构建索引"""
        for e in self.evidence:
            perf = e.get('performance', {})
            dim = perf.get('dimension_id')
            subdim = perf.get('subdimension_id')

            if dim:
                self.perf_idx[dim].append(e)
            if subdim and subdim != 'PRS_NA':
                self.subdim_idx[subdim].append(e)

        print(f"   Index: {len(self.perf_idx)} dimensions, {len(self.subdim_idx)} subdimensions")

    def retrieve(self, dimensions, subdimensions=None):
        """根据dimensions和subdimensions检索相关Evidence"""
        evds = []

        # 按dimension检索
        for d in dimensions:
            for e in self.perf_idx.get(d, []):
                if e not in evds:
                    evds.append(e)

        # 按subdimension补充检索
        if subdimensions:
            for sd in subdimensions:
                for e in self.subdim_idx.get(sd, []):
                    if e not in evds:
                        evds.append(e)

        print(f"🔍 Retrieved: {len(evds)} Evidence records")
        return evds

kb = KnowledgeBase(CONFIG['evidence_path'])

📚 Evidence loaded: 320 records
   Index: 8 dimensions, 53 subdimensions


In [ ]:
class Codebook:
    """加载Appendix代码本"""

    def __init__(self, path):
        self.data = {}
        if Path(path).exists():
            self.data = json.load(open(path, encoding='utf-8'))
            print(f"📖 Codebook loaded: {len(self.data)} tables")

    def subset(self, max_chars=40000):
        """提取用于prompt的子集"""
        priority = [
            'A_indicators', 'A_categories',
            'C_performance', 'C_subdimensions', 'C_outcome_types',
            'B_methods', 'B_units', 'B_data_sources',
            'D_directions', 'D_significance',
            'E_settings', 'E_countries',
            'K_climate', 'F_quality'
        ]
        out, sz = {}, 0
        for n in priority:
            if n in self.data:
                simplified = {}
                for code, entry in self.data[n].items():
                    simplified[code] = {
                        "name": entry.get('name', code),
                        "definition": entry.get('definition', '')[:200]
                    }
                s = len(json.dumps(simplified, ensure_ascii=False))
                if sz + s < max_chars:
                    out[n] = simplified
                    sz += s
        return out

codebook = Codebook(CONFIG['appendix_path'])

📖 Codebook loaded: 34 tables


In [ ]:
qpath = Path(CONFIG['user_query_path'])
if qpath.exists():
    USER_QUERY = json.load(open(qpath, encoding='utf-8'))
    print(f"✅ Loaded: {qpath.name}")
else:
    print("📤 Please upload user query JSON:")
    up = files.upload()
    USER_QUERY = json.loads(list(up.values())[0].decode('utf-8'))

# 显示项目信息
print(f"📋 Project: {USER_QUERY.get('project', {}).get('name', 'N/A')}")
print(f"   Location: {USER_QUERY.get('project', {}).get('location', 'N/A')}")
print(f"   Scale: {USER_QUERY.get('project', {}).get('scale', 'N/A')}")
print(f"   Phase: {USER_QUERY.get('project', {}).get('phase', 'N/A')}")

# 显示性能查询信息
perf_query = USER_QUERY.get('performance_query', {})
print(f"\n🎯 Performance Query:")
print(f"   Dimensions: {perf_query.get('dimensions', [])}")
print(f"   Subdimensions: {perf_query.get('subdimensions', [])}")

# 显示空间区域信息（如果有）
zones = USER_QUERY.get('spatial_zones', [])
if zones:
    print(f"\n🗺️ Spatial Zones ({len(zones)}):")
    for z in zones:
        print(f"   • {z.get('zone_id')}: {z.get('zone_name')} ({z.get('status', 'N/A')})")

✅ Loaded: GreenSVC-AI_mock_filled_query_single_performance_photos_45_per_zone.json
📋 Project: Amsterdam Pocket Park Retrofit (Oost District)
   Location: Amsterdam, Netherlands (Oost district, near Oosterpark)
   Scale: S
   Phase: renovation

🎯 Performance Query:
   Dimensions: ['PRF_BEH']
   Subdimensions: ['PRS_WLK', 'PRS_NAV']

🗺️ Spatial Zones (6):
   • zone_0: North Entrance Forecourt (moderate)
   • zone_1: Central Multi-use Lawn (good)
   • zone_2: Family Play Pocket (poor)
   • zone_3: Woodland Edge Loop Path (poor)
   • zone_4: Waterfront Deck & Seating (critical)
   • zone_5: Service Node (Toilet/Kiosk) (moderate)


## 4. Retrieve & Build Prompt

In [ ]:
# 获取dimensions和subdimensions
perf_query = USER_QUERY.get('performance_query', {})
dims = perf_query.get('dimensions', [])
subdims = perf_query.get('subdimensions', [])

print(f"🎯 Target dimensions: {dims}")
print(f"🎯 Target subdimensions: {subdims}")

# 检索相关证据
matched_evd = kb.retrieve(dims, subdims)

# 获取Codebook子集
cb_subset = codebook.subset(CONFIG['max_codebook_chars'])
print(f"📖 Codebook subset: {len(cb_subset)} tables")

🎯 Target dimensions: ['PRF_BEH']
🎯 Target subdimensions: ['PRS_WLK', 'PRS_NAV']
🔍 Retrieved: 66 Evidence records
📖 Codebook subset: 8 tables


In [ ]:
PROMPT_TEMPLATE = """
# GreenSVC-AI Stage 1: Evidence-Based Indicator Matching

## System Role
You are an evidence-based landscape design consultant. Your task is to recommend environmental indicators for landscape projects based on scientific evidence.

## Core Principles
1. **Evidence-Based**: Every recommendation must cite evidence_id
2. **Code Expansion**: Use Codebook to expand all codes to code + name + definition
3. **No Fabrication**: Only use evidence that exists in the input

---

## Input Data

### User Query
```json
{query}
```

### Codebook (Code Definitions)
```json
{codebook}
```

### Evidence ({evd_count} records)
```json
{evidence}
```

---

## Output Format

Output a JSON object with this structure:

```json
{{
  "metadata": {{
    "project_name": "from query.project.name",
    "target_dimensions": [{{"code": "PRF_BEH", "name": "...", "definition": "..."}}],
    "target_subdimensions": [{{"code": "PRS_WLK", "name": "...", "definition": "..."}}],
    "evidence_used": 10
  }},

  "recommended_indicators": [
    {{
      "rank": 1,
      "indicator": {{
        "code": "IND_XXX",
        "name": "from Codebook",
        "definition": "from Codebook",
        "formula": "from Evidence",
        "category": {{"code": "CAT_XXX", "name": "...", "definition": "..."}}
      }},
      "performance": {{
        "dimension": {{"code": "PRF_BEH", "name": "...", "definition": "..."}},
        "subdimension": {{"code": "PRS_WLK", "name": "...", "definition": "..."}},
        "outcome_measure": "from Evidence",
        "outcome_type": {{"code": "OUT_XXX", "name": "...", "definition": "..."}}
      }},
      "evidence": [
        {{
          "evidence_id": "EVD_xxx",
          "citation": "Author (Year). Title.",
          "year": 2024,
          "doi": "10.xxx",
          "relationship": {{
            "direction": {{"code": "DIR_POS", "name": "Positive", "definition": "..."}},
            "effect_size": "0.45",
            "p_value": "<0.001",
            "significance": {{"code": "SIG_001", "name": "...", "definition": "..."}}
          }},
          "study": {{
            "design": {{"code": "DES_CRS", "name": "...", "definition": "..."}},
            "sample_size": 100,
            "setting": {{"code": "SET_PRK", "name": "...", "definition": "..."}},
            "country": {{"code": "CNT_NLD", "name": "...", "definition": "..."}}
          }},
          "quality": {{
            "tier": {{"code": "TIR_T2", "name": "...", "definition": "..."}},
            "confidence": {{"code": "CON_HIG", "name": "...", "definition": "..."}}
          }}
        }}
      ],
      "measurement": {{
        "method": {{"code": "MTH_XXX", "name": "...", "definition": "..."}},
        "unit": {{"code": "UNT_XXX", "name": "...", "definition": "..."}},
        "data_source": {{"code": "SRC_XXX", "name": "...", "definition": "..."}}
      }},
      "target_direction": {{
        "direction": "INCREASE or DECREASE",
        "derivation": "Based on evidence direction and performance goal"
      }},
      "rationale": "Why this indicator is recommended"
    }}
  ],

  "indicator_relationships": [
    {{
      "indicators": [{{"code": "IND_A", "name": "..."}}, {{"code": "IND_B", "name": "..."}}],
      "type": "SYNERGISTIC/INVERSE/INDEPENDENT",
      "explanation": "How they interact"
    }}
  ],

  "summary": {{
    "total_indicators": 5,
    "total_evidence": 12,
    "key_findings": ["Finding 1", "Finding 2"],
    "evidence_gaps": ["Gap 1"]
  }}
}}
```

## Rules
1. Recommend 5-8 indicators relevant to target dimensions/subdimensions
2. Each indicator MUST have evidence_id from the provided Evidence
3. Expand ALL codes using Codebook
4. Do NOT output numerical target values
5. Output valid JSON directly, no markdown code blocks
"""

def build_prompt(query, evidence, cb):
    return PROMPT_TEMPLATE.format(
        query=json.dumps(query, ensure_ascii=False, indent=2),
        codebook=json.dumps(cb, ensure_ascii=False, indent=2),
        evidence=json.dumps(evidence[:60], ensure_ascii=False, indent=2),
        evd_count=len(evidence)
    )

prompt = build_prompt(USER_QUERY, matched_evd, cb_subset)
print(f"📝 Prompt: {len(prompt):,} chars (~{len(prompt)//4:,} tokens)")

📝 Prompt: 259,955 chars (~64,988 tokens)


## 5. Call API

In [ ]:
class Runner:
    def __init__(self, model):
        self.model = genai.GenerativeModel(model)
        print(f"✅ Model: {model}")

    def run(self, prompt):
        print(f"🚀 Calling API (~{len(prompt)//4:,} tokens)")
        cfg = genai.GenerationConfig(temperature=0.2, max_output_tokens=32768)
        resp = self.model.generate_content(prompt, generation_config=cfg)
        text = resp.text
        print(f"✅ Response: {len(text):,} chars")
        if not text.rstrip().endswith('}'): print("⚠️ May be truncated")
        return self._parse(text)

    def _parse(self, text):
        text = text.strip()
        for p in ['```json', '```']:
            if text.startswith(p): text = text[len(p):]
        if text.endswith('```'): text = text[:-3]
        text = text.strip()
        try: return json.loads(text)
        except: pass
        if not text.endswith('}'):
            r = text.rstrip().rstrip(',: \n\t"')
            r += ']' * (text.count('[') - text.count(']'))
            r += '}' * (text.count('{') - text.count('}'))
            try:
                res = json.loads(r)
                res['_warning'] = 'Auto-repaired'
                return res
            except: pass
        return {'raw': text, 'error': True}

runner = Runner(CONFIG['model_name'])

✅ Model: gemini-3-pro-preview


In [ ]:
print("="*60)
print("🌿 GreenSVC-AI Stage 1")
print("="*60)
result = runner.run(prompt)
print("="*60)

🌿 GreenSVC-AI Stage 1
🚀 Calling API (~64,988 tokens)
✅ Response: 27,503 chars
⚠️ May be truncated


## 6. View Results

In [ ]:
if 'error' not in result:
    if '_warning' in result: print(f"⚠️ {result['_warning']}")

    print("\n📊 Results Summary")
    print("="*60)

    meta = result.get('metadata', {})
    print(f"Project: {meta.get('project_name', 'N/A')}")
    print(f"Evidence used: {meta.get('evidence_used', 0)} records")

    # 目标维度
    dims = meta.get('target_dimensions', [])
    if dims:
        print(f"\n🎯 Target Dimensions: {[d.get('code') for d in dims]}")

    subdims = meta.get('target_subdimensions', [])
    if subdims:
        print(f"🎯 Target Subdimensions: {[s.get('code') for s in subdims]}")

    # 推荐指标
    inds = result.get('recommended_indicators', [])
    print(f"\n🎯 Recommended Indicators ({len(inds)}):")
    for ind in inds:
        i = ind.get('indicator', {})
        td = ind.get('target_direction', {})
        evs = ind.get('evidence', [])

        print(f"\n  {ind.get('rank', '?')}. {i.get('code', 'N/A')} - {i.get('name', 'N/A')}")
        print(f"     Direction: {td.get('direction', 'N/A')}")
        if evs:
            print(f"     Evidence: {evs[0].get('evidence_id', 'N/A')}")

    # 关键发现
    summary = result.get('summary', {})
    findings = summary.get('key_findings', [])
    if findings:
        print("\n📝 Key Findings:")
        for f in findings[:3]:
            print(f"   • {f[:80]}...")

else:
    print(f"❌ Error: {result.get('error', 'Unknown')}")


📊 Results Summary
Project: Amsterdam Pocket Park Retrofit (Oost District)
Evidence used: 7 records

🎯 Target Dimensions: ['PRF_BEH']
🎯 Target Subdimensions: ['PRS_WLK', 'PRS_SAF_VIS']

🎯 Recommended Indicators (5):

  1. IND_GVI - Green View Index
     Direction: INCREASE
     Evidence: EVD_Lu2018_2

  2. IND_VAC - Visible Accessibility Index
     Direction: INCREASE
     Evidence: EVD_Larkin2021_1

  3. IND_VPE - Vegetation Permeability Index
     Direction: INCREASE
     Evidence: EVD_Li2023_2

  4. IND_WLK_IDX - Walkability Index (Visual)
     Direction: INCREASE
     Evidence: EVD_Zhang2024_8

  5. IND_SVF - Sky View Factor
     Direction: INCREASE
     Evidence: EVD_Zou2024_1

📝 Key Findings:
   • Green View Index (GVI) is the strongest predictor for walking behavior and gener...
   • Visible Accessibility (VAC) and Walkability Index (WLK_IDX) are critical for the...
   • Vegetation Permeability (VPE) is a crucial 'check' indicator to ensure the shade...


In [ ]:
print("📄 Full JSON:")
print(json.dumps(result, ensure_ascii=False, indent=2))

📄 Full JSON:
{
  "metadata": {
    "project_name": "Amsterdam Pocket Park Retrofit (Oost District)",
    "target_dimensions": [
      {
        "code": "PRF_BEH",
        "name": "Use, Accessibility & Safety",
        "definition": "Outcomes related to use patterns and behaviors, including functional convenience/accessibility and perceived safety that shape use."
      }
    ],
    "target_subdimensions": [
      {
        "code": "PRS_WLK",
        "name": "Walkability Score",
        "definition": "A performance metric assessing the degree to which an environment is friendly to walking. It synthesizes variables such as residential density, intersection connectivity, land-use mix, and safety infrastructure."
      },
      {
        "code": "PRS_SAF_VIS",
        "name": "Performance - Visual Safety",
        "definition": "Measures the perception of safety derived specifically from visual cues within an environment. This performance indicator assesses how elements like lighting brigh

In [ ]:
out_dir = Path(CONFIG['output_path'])
out_dir.mkdir(parents=True, exist_ok=True)
ts = datetime.now().strftime('%Y%m%d_%H%M%S')
out_file = out_dir / f'STAGE1_{ts}.json'
json.dump(result, open(out_file, 'w', encoding='utf-8'), ensure_ascii=False, indent=2)
print(f"✅ Saved: {out_file}")

✅ Saved: /content/drive/MyDrive/GreenSVC-AI-paper/Outputs/STAGE1_20251225_001327.json
